# clean_midi Matrixes Search (ver. 2.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (Setup Environment)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools
!pip install sklearn
!pip install tqdm

!unzip /content/tegridy-tools/tegridy-data/clean_midi_matrixes.zip

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
from tqdm import tqdm

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')
import TMIDIX

from sklearn.metrics import pairwise_distances

os.chdir('/content/')
print('Done!')

In [ ]:
#@title Load clean_midi matrixes pickle file
data = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/clean_midi_matrixes')

In [ ]:
#@title Search
full_path_to_MIDI_file = "/content/tegridy-tools/tegridy-tools/seed2.mid" #@param {type:"string"}
print('=' * 70)
print('Loading MIDI file...')

score = TMIDIX.midi2ms_score(open(full_path_to_MIDI_file, 'rb').read())

events_matrix = []

itrack = 1

while itrack < len(score):
    for event in score[itrack]:         
      events_matrix.append(event)
    itrack += 1

# Sorting...
events_matrix.sort(key=lambda x: x[1])

# recalculating timings
for e in events_matrix:
    e[1] = int(e[1] / 10)
    if e[0] == 'note':
      e[2] = int(e[2] / 20)

# final processing...

melody_chords = []

patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

pe = events_matrix[0]
for e in events_matrix:

  if e[0] == 'note':
    # ['note', start_time, duration, channel, note, velocity]
    time = max(0, min(255, e[1]-pe[1]))
    duration = max(1, min(255, e[2]))
    channel = max(0, min(15, e[3]))

    if e[3] != 9:
      instrument = max(0, min(127, patches[e[3]]))
    else:
      instrument = max(128, min(255, patches[e[3]]+128))

    if e[3] != 9:

      pitch = max(1, min(127, e[4]))
    else:
      pitch = max(129, min(255, e[4]+128))

    if e[3] != 9:
      velocity = max(1, min(127, e[5]))
    else:
      velocity = max(129, min(255, e[5]+128))

    melody_chords.append([time, duration, channel, instrument, pitch, velocity])

  if e[0] == 'patch_change':
    # ['patch_change', dtime, channel, patch]
    time = max(0, min(127, e[1]-pe[1]))
    channel = max(0, min(15, e[2]))
    patch = max(0, min(127, e[3]))

    patches[channel] = patch

  pe = e # Previous event

MATRIX = [[0]*256 for i in range(38)]

for m in melody_chords:

  MATRIX[0][m[0]] += 1
  MATRIX[1][m[1]] += 1
  MATRIX[2][m[2]] += 1 
  MATRIX[3][m[3]] += 1
  MATRIX[4][m[4]] += 1
  MATRIX[5][m[5]] += 1
  MATRIX[m[2]+6][m[3]] += 1
  MATRIX[m[2]+22][m[4]] += 1

print('Dones!')
print('=' * 70)

print('Searching...Please wait...')

scores = []

for D in tqdm(data):
    dist = pairwise_distances(MATRIX, 
                             D[2],
                              )[0][0]
                              
    scores.append(dist)
    if dist == 0:
      break

print('Done!')
print('=' * 70)
    
print('Euclidian distance ==', min(scores))
print('=' * 70)
print('Artist:', data[scores.index(min(scores))][0])
print('Composition Name:', data[scores.index(min(scores))][1])
print('=' * 70)

# Congrats! You did it! :)